In [106]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.signal
from scipy.io.wavfile import write

In [553]:
SF = 7 #Spreading Factor
B = 125e3 # Bandwidth
fmin = 433.05e6
fmax = 434.79e6 - B

Nsym = 2**SF
spb = 120e-3/(64*8*7) # Seconds per bit derived from rate table
Ts = spb*2**SF # Duration of a chirp
print(Ts)

sn = 4 # Samples per period of highest frequency (>= 2 for nyquist)
fs = int(sn*(fmax+B)) # resolution

0.004285714285714286


$x(t) = \text{sin} \left[ 2 \pi t \left( \frac k2 t + f_0 \right) \right]$

In [554]:
chirp = lambda f0, k, t: np.sin(2*np.pi*t*(f0 + k*t/2))
shift = lambda x, t: np.concatenate((x[int(t*len(x)):], x[:int(t*len(x))])) # used for generating symbols from basechirp
butter_bandpass = lambda low, high, fs, order: scipy.signal.butter(order, [2*low/fs, 2*high/fs], btype='band', output='sos')
butter_lowpass = lambda high, fs, order: scipy.signal.butter(order, 2*high/fs, btype='low', output='sos')

In [555]:
baseChirp = chirp(fmin, B/Ts, np.linspace(0, Ts, int(fs*Ts)))
downChirp = chirp(fmin+B, -B/Ts, np.linspace(0, Ts, int(fs*Ts)))
                                             
timingOffsets = np.arange(0, Nsym)/(Nsym) # times where symbols are cut and stitched together

In [600]:
# ASCII-Werte für Nachricht generieren
seq = [ord(char) for char in "Computer Networks"]

# generate and transmit signal
preamble = np.concatenate([baseChirp for i in range(0, 8)]) 
preamble = np.concatenate([preamble, downChirp, downChirp, downChirp[:len(downChirp)//2]])
postamble = np.concatenate([downChirp for i in range(0, 2)])
message = np.concatenate([shift(baseChirp, timingOffsets[i]) for i in seq])
tx = shift(baseChirp, timingOffsets[32]) #np.concatenate((preamble, message, postamble))

In [601]:
# filter carrier range
sos = butter_bandpass(fmin, fmax+B, fs, 5)
fcx = scipy.signal.sosfilt(sos, tx)

Fcx = np.abs(np.fft.rfft(fcx))
dFcarrier = np.diff(Fcx)
Bh = Fcarrier - np.max(Fcx)/2
crossings = np.where(np.sign(Bh[:-1]) != np.sign(Bh[1:]))[0]
fcarrier = int(np.mean(crossings))/Ts
fbase = fcarrier - B/2

# shift to baseband
bx = fcx * np.exp(-1j * 2 * np.pi * fbase * np.linspace(0, len(fcx)/fs, len(fcx)))

# filter base band
sos = butter_lowpass(B, fs, 5)
fbx = scipy.signal.sosfilt(sos, np.real(bx)+np.imag(bx))

In [606]:
# downsample
nsam = int(((sn*B)//Nsym + 1) * Nsym)
dsx = np.take(fbx, np.linspace(0, len(fbx)-1, nsam).astype('int'))
sos = butter_lowpass(B, nsam, 5)
fdsx = scipy.signal.sosfilt(sos, dsx)

# dechirp
downChirp = chirp(B, -(B/Ts), np.linspace(0, Ts, nsam))
dcx = fdsx * downChirp

Fdcx = np.abs(np.fft.fft(dcx))
print(len(Fdcx), B)
Fl = np.abs(Fdcx[:len(Fdcx)//B])
Fh = np.abs(Fdcx[len(Fl):2*len(Fl)])
F = Fl**2 + Fh**2

print(len(Fl))

s = round(np.argmax(F) * (Nsym/len(F)))
print(s)

plt.plot(np.abs(Fl))
plt.plot(np.abs(Fh))
#plt.plot(np.abs(F))

500096 125000.0


TypeError: slice indices must be integers or None or have an __index__ method